In [82]:
import pdfplumber
import nltk 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 
from nltk.tokenize import sent_tokenize
import faiss
from sentence_transformers import SentenceTransformer

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
import torch
import time
import re 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\prish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Search through Tf-IDF

In [21]:
pdf_path = "book1.pdf"

pages = []
pdf = pdfplumber.open(pdf_path)

for i in tqdm(pdf.pages):
    text = i.extract_text() or ""
    pages.append(text)

paragraphs = [] 

for page in pages:
    paras = re.split(r"\n\s*\n", page) #Assuming every paragraph has 2 newline spaces separating it
    for i in paras:
        i.strip()
        paragraphs.append(i)

("Number of paragraphs are ",len(paragraphs))


100%|██████████| 228/228 [00:14<00:00, 15.70it/s]


('Number of paragraphs are ', 228)

In [22]:
from nltk.corpus import stopwords 
stop_words = stopwords.words("english")

vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    max_df=0.90, #Removes overly common (non-discriminative terrms)
    min_df=2 # Removes extremely rare terms (noise)
)

tfidf_matrix = vectorizer.fit_transform(paragraphs)
print("Tfidf Matrix shape is ", tfidf_matrix.shape)

Tfidf Matrix shape is  (228, 3150)


* Sparse Matrix of 228 rows (paragraphs) and 3150 feature matrix created for every paragraph ❔

In [54]:
query = "Lord Voldermort"
top_k = 4 

query_vector = vectorizer.transform([query]) #Generates a vec embedding of same shape as the tdidf one 
scores = cosine_similarity(query_vector, tfidf_matrix)[0]

idx = np.argsort(scores)[::-1][:top_k] #argsort returns the indexes of the elements according to their values (score) in ascending order and reverse -> top 5

for i in idx:
    print(f"Paragraph : {paragraphs[i][0:200]} \nScores : {scores[i]}\n")

Paragraph : When a letter arrives for unhappy but
ordinary Harry Potter, a decade-old secret
is revealed to him. His parents were
wizards, killed by a Dark Lord’s curse
when Harry was just a baby, and which he
so 
Scores : 0.12645905029115623

Paragraph : 210 Harry Potter
Hallowe’en like that, for all I knew you’d seen me coming to look
at what was guarding the Stone.’
‘You let the troll in?’
‘Certainly. I have a special gift with trolls – you must hav 
Scores : 0.08098958607063725

Paragraph : 54 Harry Potter
out, Harry wouldn’t have noticed it was there. The people hurry-
ing by didn’t glance at it. Their eyes slid from the big book shop
on one side to the record shop on the other as if th 
Scores : 0.08085259435317048

Paragraph : The Man with Two Faces 211
instructions – he is a great wizard and I am weak –’
‘You mean he was there in the classroom with you?’ Harry
gasped.
‘He is with me wherever I go,’ said Quirrell quietly. ‘ 
Scores : 0.07750553802544466



* Need to compute the *precision@k*, the ground truth required is not present with me currently so keeping it on hold 📓

# Embedding Similarity Search

In [56]:
pdf_path = "book1.pdf"

pages = []
pdf = pdfplumber.open(pdf_path)

for i in tqdm(pdf.pages):
    text = i.extract_text() or ""
    pages.append(text)
print("Number of pages are ",len(pages))

100%|██████████| 228/228 [00:15<00:00, 14.89it/s]

Number of pages are  228


In [66]:
sents_per_chunk = 6 
sent_overlap = 2

all_sentences = []
page_of_sentence = [] 

for page_index, page in enumerate(pages, start=1):
    for s in sent_tokenize(page):
        s = s.strip()
        all_sentences.append(s)
        page_of_sentence.append(page_index)

print("Total sentences ", len(all_sentences))

#Chunks with overlap
chunks = []
chunk_meta = []

i = 0
while i < len(all_sentences):
    start = i 
    end = min(i + sents_per_chunk, len(all_sentences))
    chunk_text = " ".join(all_sentences[start:end])
    chunks.append(chunk_text)
    chunk_meta.append({
        "start_sent" : start,
        "end_sent" : end-1,
        "start_page":page_of_sentence[start],
        "end_page": page_of_sentence[end-1]
    })
    i += sents_per_chunk - sent_overlap

print(f"Built {len(chunks)} chunks")

Total sentences  5085
Built 1272 chunks


In [72]:
model_id = "KaLM-Embedding/KaLM-embedding-multilingual-mini-instruct-v2.5"
model_arguments = {"torch_dtype":torch.bfloat16,
                   "device_map":"cuda"
                   #"attn_implementation":"flash_attention_2"
                   }

model = SentenceTransformer(model_id,
                            cache_folder = "models",
                            model_kwargs = model_arguments)

`torch_dtype` is deprecated! Use `dtype` instead!


In [77]:
#Batch speeds encoding and measure embedding time for profiling

batch = 16 
num_chunks = len(chunks)
embeddings = np.zeros((num_chunks, model.get_sentence_embedding_dimension()), dtype="float32") #There are 1272 chunks and this model's embedding is 1,896 -> (1272,896)

t0 = time.perf_counter() 
for i in tqdm(range(0, num_chunks, batch)):
    batch_texts = chunks[i:i+batch]
    embs = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False, batch_size=batch)
    embeddings[i:i+len(embs), :] = embs 
t1 = time.perf_counter()

print(f"Total {num_chunks} chunks embedded in {t1-t0:.3f}s \n Average {(t1-t0)/num_chunks:.4f}s per chunk")


100%|██████████| 80/80 [00:17<00:00,  4.65it/s]

Total 1272 chunks embedded in 17.217s 
 Average 0.0135s per chunk


In [78]:
norms = np.linalg.norm(embs, keepdims=True, axis=1)
norms[norms==0] = 1.0
embeddings_norm = embs/norms 
embeddings_norm = embeddings_norm.astype("float32")

# Langchain flow alternative

In [96]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.schema import Document

import json, os

In [102]:
pdf_path = "book1.pdf"
pdf = PyPDFLoader(pdf_path)
docs = pdf.load() 
print("Number of loaded documents are ", len(docs))
print("Example of one document \n ", docs[35].page_content[:250])

Number of loaded documents are  228
Example of one document 
  34 Harry Potter  
 
been trying to do. He shouted at Harry for about half an hour and 
then told him to go and make a cup of tea. Harry shuffled miser-
ably off into the kitchen, and by the time he got back, the post  
had arrived, right into Uncle V


In [103]:
chunk_size = 600 
chunk_overlap = 150 

splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                          chunk_overlap=chunk_overlap,
                                          separators=["\n\n", "\n", " ", ""]
                                          )
chunked_docs = splitter.split_text(docs)
print("Chunks created ", len(chunked_docs))


TypeError: expected string or bytes-like object, got 'list'

In [104]:
docs

[Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2012-05-01T23:57:27+10:00', 'subject': "When a letter arrives for unhappy but ordinary Harry Potter, a decade-old secret is revealed to him. His parents were wizards, killed by a Dark Lord's curse when Harry was just a baby, and which he somehow survived. Escaping from his unbearable Muggle guardians to Hogwarts, a wizarding school brimming with ghosts and enchantments, Harry stumbles into a sinister adventure when he finds a threeheaded dog guarding a room on the third floor. Then he hears of a missing stone with astonishing powers which could be valuable, dangerous, or both.", 'author': 'J.K. Rowling', 'keywords': '', 'moddate': '2012-07-01T14:20:39+10:00', 'title': 'Harry Potter and the Philosopher’s Stone', 'source': 'book1.pdf', 'total_pages': 228, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'Acrobat Distiller 7.0.5 (Wind